In [1]:
import pandas as pd

In [2]:
# import the dataset
synthetic_df = pd.read_csv("../../data/bronze/synthetic_data.csv")

In [4]:
synthetic_df["id"] = synthetic_df.index

In [15]:
synthetic_df["num_procedures_bin"] = synthetic_df["num_procedures"].apply(lambda x: False if (x==0) else True)

In [21]:
synthetic_df["num_medications_perday"] = synthetic_df["num_medications"]/synthetic_df["time_in_hospital"]

In [47]:
# synthetic_df["number_outpatient_perday"] = synthetic_df["number_outpatient"]/synthetic_df["time_in_hospital"]
# synthetic_df["number_inpatient_perday"] = synthetic_df["number_inpatient"]/synthetic_df["time_in_hospital"]

In [43]:
bin_labels_4 = ['Normal', 'Mild', 'Moderate', 'Severe']
synthetic_df['time_in_hospital_severitylvl'] = pd.qcut(synthetic_df['time_in_hospital'], q=[0, 0.25, 0.5, 0.75, 1], labels=bin_labels_4)

In [ ]:
bin_labels_4 = ['Normal', 'Mild', 'Moderate', 'Severe']
synthetic_df['time_in_hospital_severitylvl'] = pd.qcut(synthetic_df['time_in_hospital'], q=[0, 0.25, 0.5, 0.75, 1], labels=bin_labels_4)

In [48]:
synthetic_df["number_emergency"]

0         1
1         0
2         0
3        12
4         0
         ..
78436     1
78437     0
78438     0
78439     0
78440     0
Name: number_emergency, Length: 78441, dtype: int64

In [49]:
synthetic_df["number_emergency"].apply(lambda x: "normalPatient" if (x==0) else x)

0                    1
1        normalPatient
2        normalPatient
3                   12
4        normalPatient
             ...      
78436                1
78437    normalPatient
78438    normalPatient
78439    normalPatient
78440    normalPatient
Name: number_emergency, Length: 78441, dtype: object

In [53]:
synthetic_df["age"][0]

'[60-70)'